In [1]:
from pathlib import Path
from openai import OpenAI
client = OpenAI()

speech_file_path = Path(__file__).parent / "speech.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input="Today is a wonderful day to build something people love!"
)

response.stream_to_file(speech_file_path)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [1]:
from transformers import VitsModel, AutoTokenizer
import torch

model = VitsModel.from_pretrained("facebook/mms-tts-eng")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-eng")

text = "some example text in the English language"
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight

In [12]:
model = VitsModel.from_pretrained("facebook/mms-tts-nan")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-nan")

# Rest of your code remains the same (text processing, input creation, etc.)
text = "some example text in the English language"
inputs = tokenizer(text, return_tensors="pt")

with torch.no_grad():
    output = model(**inputs).waveform


Some weights of the model checkpoint at facebook/mms-tts-nan were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

In [13]:
from IPython.display import Audio

Audio(output.numpy(), rate=model.config.sampling_rate)


In [ ]:
##API references for intergration 


from flask import Flask, request, jsonify
import torch
from transformers import VitsModel, AutoTokenizer

app = Flask(__name__)

# Load the pre-trained model and tokenizer (outside the request handler for efficiency)
model = VitsModel.from_pretrained("facebook/mms-tts-eng")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-eng")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the appropriate device
model.eval()  # Set the model to evaluation mode (no dropout)

@app.route("/synthesize", methods=["POST"])
def synthesize():
    try:
        # Get text from the request body
        text = request.json["text"]
        if not text:
            return jsonify({"error": "Missing 'text' field in request body"}), 400

        # Tokenize the text
        inputs = tokenizer(text, return_tensors="pt").to(device)

        # Synthesize speech with torch.no_grad for efficiency
        with torch.no_grad():
            waveform = model(**inputs).waveform.cpu().numpy().squeeze()

        # Handle potential errors (e.g., GPU memory issues)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

    # Return the synthesized waveform as a base64-encoded string
    waveform_b64 = base64.b64encode(waveform).decode("utf-8")
    return jsonify({"waveform": waveform_b64})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)  # Run the Flask app on all interfaces


In [ ]:
## JS part 

function synthesizeText(text) {
    fetch("/synthesize", {
        method: "POST",
        headers: { "Content-Type": "application/json" },
        body: JSON.stringify({ text: text })
    })
    .then(response => response.json())
    .then(data => {
        if (data.error) {
            // Handle errors
            console.error(data.error);
            return;
        }

        const waveformB64 = data.waveform;
        // Use the waveformB64 string to play the synthesized speech
        // (Code for audio playback omitted for brevity)
    });
}


In [ ]:
## Speech To text using Google API - easy to use 


https://cloud.google.com/speech-to-text/docs/speech-to-text-requests 